# Fitek API data pipeline

## Get data from API request

In [1]:
#Import modules
import requests
import json
import uuid
import pandas as pd
import numpy as np
from datetime import timedelta, date
from bs4 import BeautifulSoup as bs
import pyodbc
from sqlalchemy import create_engine
import urllib

Current ETL setup is designed to upload all previous day invoices to SQL Server

In [ ]:
#Set the dates logic
date_today = date.today()
date_tomorrow_1 = date.today() + timedelta(days=1)
date_yesterday_1 = date.today() - timedelta(days=1)
date_tomorrow_2 = date.today() + timedelta(days=2)
date_yesterday_2 = date.today() - timedelta(days=2)

#print(date_yesterday_1.strftime('%Y-%m-%d'))
#print(date_yesterday_1.strftime('%Y-%m-%d'))
#print(date_yesterday_2.strftime('%Y-%m-%d'))
#print(date_today.strftime('%Y-%m-%d'))

In case there are memory issues when loading API data, type in "jupyter notebook --NotebookApp.iopub_data_rate_limit=1000000000" in your command line

In [ ]:
#Login details
authorization_token = "" #Enter authorisation token provided
integrator_id = "" #Enter integrator id provided

#Generate request id
request_id = str(uuid.uuid4())

#Date selection
invoice_date_greater_than_equal_UTC = date_yesterday_1.strftime('%Y-%m-%d')
invoice_date_less_than_equal_UTC = date_yesterday_1.strftime('%Y-%m-%d')
invoice_date_greater_than_UTC = date_yesterday_2.strftime('%Y-%m-%d')
invoice_date_less_than_UTC = date_today.strftime('%Y-%m-%d')

#Loading the request
url = "https://fitekin.com/ExportApi/Invoices.v3"

payload = "{\r\n\"AuthorizationToken\":\""+authorization_token+"\", // Autorizacijos raktas\r\n  \"IntegratorId\": \""+integrator_id+"\", // Integratorius\r\n  \"RequestId\": \""+request_id+"\", // Random generuojamas ID \r\n  \"Extensions\": [], //\r\n  \"QueryParams\": { //\r\n    \"Debug\": false, // Jei debug false - saskaita taps eksportuota | jei true - liks eksportuojama busenos\r\n    \"InvoiceStatus\": [ // Pasirinkti kokias saskaitas noresite eksportuoti (pagal busenas)\r\n       //\"New\", // Nauja\r\n       //\"InApproval\", //Patvirtinimo sekoje\r\n       //\"NotForExport\", //Neeksportuojama\r\n       //\"PendingExport\", //Eksportuojama\r\n       \"Exported\" //Eksportuota (jei debug-false tai tampa eksportuota)\r\n       //\"Deleted\", // Istrinta\r\n       //\"Rejected\" // Atsakta\r\n    ],\r\n    \"InvoiceDateGreaterThanEqualUTC\": \""+invoice_date_greater_than_equal_UTC+"\", // Periodo pasirinkimai kokias saskaitas noresite gauti\r\n    \"InvoiceDateLessThanEqualUTC\": \""+invoice_date_less_than_equal_UTC+"\",\r\n    \"InvoiceDateGreaterThanUTC\": \""+invoice_date_greater_than_UTC+"\",\r\n    \"InvoiceDateLessThanUTC\": \""+invoice_date_less_than_UTC+"\"\r\n    }\r\n}\r\n"
headers = {'Content-Type': 'application/json'}

#Printing the request
response = requests.request("POST", url, headers=headers, data=payload)
response_content = response.content

#Viewing all information from XML text
soup = bs(response_content, 'xml')

## Create a dataframe from API request

In [ ]:
#Invoice tag
#Specify the invoice tag
invoices = soup.select('Invoice')
invoice_key = {}


#Fetch invoice id
invoice_ids = [invoice['invoiceId'] for invoice in invoices]
invoice_key['InvoiceId'] = invoice_ids

df_invoice_ids = pd.DataFrame(invoice_ids, columns = ['InvoiceId'])


#Fetch invoice number
invoice_numbers = [invoice.find('InvoiceNumber') for invoice in invoices]
invoice_key['InvoiceNumber'] = invoice_numbers

df_invoice_numbers = pd.DataFrame(invoice_numbers, columns = ['InvoiceNumber'])


#Fetch account id
account_ids = [invoice.find('GeneralLedger') for invoice in invoices]
invoice_key['AccountId'] = account_ids

df_account_ids = pd.DataFrame(account_ids, columns = ['AccountId'])


#Fetch invoice date
invoice_dates = [invoice.find('InvoiceDate') for invoice in invoices]
invoice_key['InvoiceDate'] = invoice_dates

df_invoice_dates = pd.DataFrame(invoice_dates, columns = ['InvoiceDate'])


#Fetch invoice amount
invoice_amounts = [invoice.find('InvoiceSum') for invoice in invoices]
invoice_key['InvoiceAmount'] = invoice_amounts

df_invoice_amounts = pd.DataFrame(invoice_amounts, columns = ['InvoiceAmount'])

#Fetch invoice VAT amount
invoice_vat_amounts = [invoice.find('TotalVATSum') for invoice in invoices]
invoice_key['TotalVATAmount'] = invoice_vat_amounts

df_invoice_vat_amounts = pd.DataFrame(invoice_vat_amounts, columns = ['TotalVATAmount'])


#Fetch invoice total amount
invoice_total_amounts = [invoice.find('TotalSum') for invoice in invoices]
invoice_key['TotalAmount'] = invoice_total_amounts

df_invoice_total_amounts = pd.DataFrame(invoice_total_amounts, columns = ['TotalAmount'])


#Fetch invoice currency
invoice_currency = [invoice.find('Currency') for invoice in invoices]
invoice_key['Currency'] = invoice_currency

df_invoice_currency = pd.DataFrame(invoice_currency, columns = ['Currency'])


#Fetch invoice description
invoice_descriptions = [invoice.find('Description') for invoice in invoices]
invoice_key['Description'] = invoice_descriptions

df_invoice_descriptions = pd.DataFrame(invoice_descriptions, columns = ['Description'])


#Fetch item unit
invoice_item_units = [invoice.find('ItemUnit') for invoice in invoices]
invoice_key['ItemUnit'] = invoice_item_units

df_invoice_item_units = pd.DataFrame(invoice_item_units, columns = ['ItemUnits'])


#Fetch item amounts
invoice_item_amounts = [invoice.find('ItemAmount') for invoice in invoices]
invoice_key['ItemAmount'] = invoice_item_amounts

df_invoice_item_amounts = pd.DataFrame(invoice_item_amounts, columns = ['ItemAmount'])


#Fetch item prices
invoice_item_prices = [invoice.find('ItemPrice') for invoice in invoices]
invoice_key['ItemPrice'] = invoice_item_prices

df_invoice_item_prices = pd.DataFrame(invoice_item_prices, columns = ['ItemPrice'])


#Fetch departments
invoice_departments = [invoice.find('PADALINYS') for invoice in invoices]
invoice_key['Department'] = invoice_departments

df_invoice_departments = pd.DataFrame(invoice_departments, columns = ['Department'])


#Fetch contract number
invoice_contract_numbers = [invoice.find('ContractNumber') for invoice in invoices]
invoice_key['ContractNumber'] = invoice_contract_numbers

df_invoice_contract_numbers = pd.DataFrame(invoice_contract_numbers, columns = ['ContractNumber'])



#Seller tag
#Specify the seller tag
sellers = soup.select('SellerParty')
seller_key = {}


#Fetch seller name
seller_names = [seller.find('Name') for seller in sellers]
seller_key['SellerName'] = seller_names

df_seller_names = pd.DataFrame(seller_names, columns = ['SellerName'])


#Fetch seller id
seller_ids = [seller.find('UniqueCode') for seller in sellers]
seller_key['SellerId'] = seller_ids

df_seller_ids = pd.DataFrame(seller_ids, columns = ['SellerId'])



#Buyer tag
#Specify the buyer tag
buyers = soup.select('BuyerParty')
buyer_key = {}


#Fetch buyer name
buyer_names = [buyer.find('Name') for buyer in buyers]
buyer_key['BuyerName'] = buyer_names

df_buyer_names = pd.DataFrame(buyer_names, columns = ['BuyerName'])

In [ ]:
#Create a dataframe from created columns
invoices_combined = [df_invoice_ids,
                 df_invoice_numbers,
                 df_invoice_contract_numbers['ContractNumber'].astype(str).str.replace('<ContractNumber>','').str.replace('</ContractNumber>','').str.replace('None',''),
                 df_account_ids['AccountId'].astype(str).str.replace('<GeneralLedger/>','').str.replace('<GeneralLedger>','').str.replace('</GeneralLedger>','').str.replace('None',''),
                 df_seller_ids.astype(str),
                 df_seller_names.astype(str),
                 df_buyer_names.astype(str),
                 df_invoice_dates,
                 df_invoice_descriptions['Description'].astype(str).str.replace('<Description>','').str.replace('</Description>','').str.replace('None',''),
                 df_invoice_departments['Department'].astype(str).str.replace('<PADALINYS>','').str.replace('</PADALINYS>','').str.replace('None',''),    
                 df_invoice_amounts,
                 df_invoice_vat_amounts,
                 df_invoice_total_amounts,
                 df_invoice_currency,
                 df_invoice_item_units['ItemUnits'].astype(str).str.replace('<ItemUnit>','').str.replace('</ItemUnit>','').str.replace('<ItemUnit/>','').str.replace('None',''),
                 df_invoice_item_amounts['ItemAmount'].astype(str).str.replace('<ItemAmount>','').str.replace('</ItemAmount>','').str.replace('<ItemAmount/>','').str.replace('None',''),
                 df_invoice_item_prices['ItemPrice'].astype(str).str.replace('<ItemPrice>','').str.replace('</ItemPrice>','').str.replace('<ItemPrice/>','').str.replace('None',''),
                 df_invoice_descriptions2]


df_invoices_all = pd.concat(invoices_combined, axis=1, join='inner')

#df_invoices_all

In [ ]:
#Add Fitek invoice link column for easier invoice access and clean the dataframe
fitek_url = 'https://fitekin.fitek.com/app/#/invoiceconfirmation/'

df_invoices_all['InvoiceLink'] = fitek_url + df_invoices_all['InvoiceId'].astype(str)
df_invoices_all_clean = df_invoices_all.replace('', np.nan)

#df_invoices_all_clean

In [ ]:
#Connect to SQL Server and load the current dataframe
server = '' #Enter server name
database = '' #Enter database name
username = '' #Enter username
password = '' #Enter password
sql_table_name = "" #Enter database table name

conn= urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password)
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn))

upload = df_invoices_all_clean.to_sql(sql_table_name, engine, schema='dbo', if_exists='append', index=False, index_label='IndexId')
upload

#Check the success of the upload
if upload == -1:
    print(f'{len(df_invoices_all_clean)} columns successfully uploaded to {sql_table_name} table')
else:
    print("Error")